In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


In [ ]:

inputs = tf.keras.Input(shape=(224, 224, 32))

# Depthwise convolution
x = layers.DepthwiseConv2D(
    kernel_size=3,
    padding="same"
)(inputs)

# Pointwise convolution
x = layers.Conv2D(
    filters=64,
    kernel_size=1,
    padding="same"
)(x)

model = tf.keras.Model(inputs, x)
model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_7              │ (None, 224, 224, 32)   │           320 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 224, 224, 64)   │         2,112 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,432 (9.50 KB)

 Trainable params: 2,432 (9.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 32))

x = layers.Conv2D(
    filters=64,
    kernel_size=3,
    padding="same",
    groups=4
)(inputs)

model = tf.keras.Model(inputs, x)
model.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 224, 224, 64)   │         4,672 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,672 (18.25 KB)

 Trainable params: 4,672 (18.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
train_dir = "/content/drive/MyDrive/DeepLearning/train"
test_dir  = "/content/drive/MyDrive/DeepLearning/test"


In [ ]:

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    shuffle=True
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    shuffle=False
)

class_names = train_ds.class_names
print(class_names)

Found 800 files belonging to 2 classes.
Found 200 files belonging to 2 classes.
['Cat', 'Dog']


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds  = test_ds.map(lambda x, y: (normalization_layer(x), y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
test_ds  = test_ds.prefetch(buffer_size=AUTOTUNE)


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(2, activation="softmax")
])


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
def train_model(model, train_ds, test_ds, epochs=10):
    history = model.fit(
        train_ds,
        validation_data=test_ds,
        epochs=epochs
    )
    return history


In [ ]:
history = train_model(model, train_ds, test_ds, epochs=10)


Epoch 1/10
 2/25 ━━━━━━━━━━━━━━━━━━━━ 1:16 3s/step - accuracy: 0.7266 - loss: 0.5913 

KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, test_ds):
    loss, acc = model.evaluate(test_ds)
    print(f"Test Accuracy: {acc*100:.2f}%")


In [ ]:
evaluate_model(model, test_ds)


7/7 ━━━━━━━━━━━━━━━━━━━━ 36s 4s/step - accuracy: 0.7940 - loss: 0.4457
Test Accuracy: 73.00%


In [ ]:
class ChannelShuffle(layers.Layer):
    def __init__(self, groups):
        super().__init__()
        self.groups = groups

    def call(self, x):
        batch, h, w, c = tf.shape(x)[0], x.shape[1], x.shape[2], x.shape[3]
        channels_per_group = c // self.groups

        x = tf.reshape(x, [batch, h, w, self.groups, channels_per_group])
        x = tf.transpose(x, [0, 1, 2, 4, 3])
        x = tf.reshape(x, [batch, h, w, c])
        return x


In [ ]:
def shufflenet_unit(x, out_channels, stride, groups):
    shortcut = x

    # 1x1 Grouped Convolution
    x = layers.Conv2D(
        out_channels, 1, padding="same", groups=groups, use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Channel Shuffle
    x = ChannelShuffle(groups)(x)

    # Depthwise Convolution
    x = layers.DepthwiseConv2D(
        3, strides=stride, padding="same", use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)

    # 1x1 Pointwise Convolution
    x = layers.Conv2D(
        out_channels, 1, padding="same", use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)

    # Residual connection
    if stride == 1 and shortcut.shape[-1] == out_channels:
        x = layers.Add()([shortcut, x])

    x = layers.ReLU()(x)
    return x


In [ ]:
def build_shufflenet(
    input_shape=(224, 224, 3),
    num_classes=2,
    groups=3
):
    inputs = layers.Input(shape=input_shape)

    # Initial layers
    x = layers.Conv2D(24, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    # ShuffleNet stages
    x = shufflenet_unit(x, 48, stride=1, groups=groups)
    x = shufflenet_unit(x, 96, stride=2, groups=groups)
    x = shufflenet_unit(x, 192, stride=2, groups=groups)

    # Classifier
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs, outputs)
    return model


In [ ]:
model = build_shufflenet()
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 112, 112, 24)   │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 112, 112, 24)   │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_8 (ReLU)                  │ (None, 112, 112, 24)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 56, 56, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 56, 56, 48)     │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 56, 56, 48)     │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_9 (ReLU)                  │ (None, 56, 56, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ channel_shuffle_3               │ (None, 56, 56, 48)     │             0 │
│ (ChannelShuffle)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_4              │ (None, 56, 56, 48)     │           432 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 56, 56, 48)     │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 56, 56, 48)     │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 56, 56, 48)     │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_10 (ReLU)                 │ (None, 56, 56, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 56, 56, 96)     │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 56, 56, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_11 (ReLU)                 │ (None, 56, 56, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ channel_shuffle_4               │ (None, 56, 56, 96)     │             0 │
│ (ChannelShuffle)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_5              │ (None, 28, 28, 96)     │           864 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 28, 28, 96)     │           384 │
│ (BatchNormalization)            │                        │             

 Total params: 64,658 (252.57 KB)

 Trainable params: 62,594 (244.51 KB)

 Non-trainable params: 2,064 (8.06 KB)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)


Epoch 1/10
 5/25 ━━━━━━━━━━━━━━━━━━━━ 1:11 4s/step - accuracy: 0.5015 - loss: 0.7578

KeyboardInterrupt: 

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
train_dataset = datasets.ImageFolder(
    root=train_dir,
    transform=train_transform
)

test_dataset = datasets.ImageFolder(
    root=test_dir,
    transform=test_transform
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Classes:", train_dataset.classes)


Classes: ['Cat', 'Dog']


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v2(weights="IMAGENET1K_V1")
model.classifier[1] = nn.Linear(model.last_channel, 2)
model = model.to(device)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 38.9MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def train(model, loader):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    return running_loss / len(loader), acc


In [ ]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    return acc


In [ ]:
epochs = 10

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader)
    test_acc = evaluate(model, test_loader)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Acc: {test_acc:.2f}%")


In [ ]:
model_shuffle = models.shufflenet_v2_x1_0(weights="IMAGENET1K_V1")
model_shuffle.fc = nn.Linear(model_shuffle.fc.in_features, 2)
model_shuffle = model_shuffle.to(device)


In [ ]:
epochs = 10

for epoch in range(epochs):
    train_loss, train_acc = train(model_shuffle, train_loader)
    test_acc = evaluate(model_shuffle, test_loader)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Acc: {test_acc:.2f}%")


In [ ]:
model_eff = models.efficientnet_b0(weights="IMAGENET1K_V1")
model_eff.classifier[1] = nn.Linear(model_eff.classifier[1].in_features, 2)
model_eff = model_eff.to(device)


In [ ]:
epochs = 10

for epoch in range(epochs):
    train_loss, train_acc = train(model_eff, train_loader)
    test_acc = evaluate(model_eff, test_loader)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Acc: {test_acc:.2f}%")
